In [9]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
# 엑셀 파일 불러오기
excel_path1 = '/Users/kyuree/Desktop/pds_capstone1/dataset/점멸신호운영현황_원본.xlsx'
xls1 = pd.ExcelFile(excel_path1)

# 각 시트를 데이터프레임으로 읽어와 리스트에 저장
always_sign = pd.read_excel(xls1, sheet_name='Table 1',header=1)
time_sign = pd.read_excel(xls1, sheet_name='Table 2',header=0)

# print(always_sign)
# print(time_sign)

##### 할 전처리
###### 1. 항시 점멸신호 -> 경찰서로 구분돼있는데 이를 자치구 명으로 바꾸어줌
###### 2. 엑셀로 변환과정에서 생긴 글자 인식 오류 -> 올바른 글자로 수정
###### 3. 운영시간 형식 수정 및 통일
###### 3-1. 평일 운영, 주말 운영 등 특징은 따로 컬럼으로 나타내주기
###### 4. 교차로별 경위도 값 추가
###### 5. 자치구별 운영 점멸신호 개수 파악


In [2]:
#1번 전처리
always_sign['경찰서'].unique()

array(['광진', '중랑', '서대문', '종로', '노원', '성동', '도봉', '종암', '성북', '용산', '은평',
       '혜화', '마포', '강북', '서부', '강서', '수서', '구로', '송파', '서초', '관악', '방배',
       '강동', '동작', '영등포'], dtype=object)

###### 대체할 값 정리,
    강서구 - 강서,
    강남구 - 수서,
    도봉구 - 도봉,
    "송파구 - 송파,
    "서대문구 - 서대문,
    "종로구 - 종로,혜화,
    "성동구 - 성동,
    "동작구 - 동작,
    "용산구 - 용산,
    "중랑구 - 중랑,
    "노원구 - 노원,
    "강동구 - 강동 , 갈동
    "구로구 - 구로
    "광진구 - 광진
    "영등포구 - 영등포
    "성북구 - 종암, 성북
    "서초구 - 방배, 서초
    "마포구 - 마포
    "은평구 - 서부, 은평
    "강북구 - 강북
    "관악구 - 관악
    양천구 - 양천
    동대문구 - 동대문,
    중구 - 남대문, 중부
    금천구 - 금천

In [56]:
#항시 점멸 - 값 대체, 새로운 컬럼 생성
for i in range(len(always_sign)):
    if always_sign.loc[i,'경찰서'] =='혜화':
        always_sign.loc[i,'자치구명'] = '종로구'
    elif always_sign.loc[i,'경찰서'] =='수서':
        always_sign.loc[i,'자치구명'] = '강남구'
    elif always_sign.loc[i,'경찰서'] =='종암':
        always_sign.loc[i,'자치구명'] = '성북구'
    elif always_sign.loc[i,'경찰서'] =='방배':
        always_sign.loc[i,'자치구명'] = '서초구'
    elif always_sign.loc[i,'경찰서'] =='서부':
        always_sign.loc[i,'자치구명'] = '은평구'
    else:
        always_sign.loc[i,'자치구명'] = always_sign.loc[i,'경찰서']+'구'
# always_sign

#시간제 점멸- 값 대체, 새로운 컬럼 생성
time_sign['경찰서'].unique()
    
for i in range(len(time_sign)):
    if time_sign.loc[i,'경찰서'] =='혜화':
        time_sign.loc[i,'자치구명'] = '종로구'
    elif time_sign.loc[i,'경찰서'] =='수서':
        time_sign.loc[i,'자치구명'] = '강남구'
    elif time_sign.loc[i,'경찰서'] =='종암':
        time_sign.loc[i,'자치구명'] = '성북구'
    elif time_sign.loc[i,'경찰서'] =='방배':
        time_sign.loc[i,'자치구명'] = '서초구'
    elif time_sign.loc[i,'경찰서'] =='서부':
        time_sign.loc[i,'자치구명'] = '은평구'
    elif time_sign.loc[i,'경찰서'] =='금천':
        time_sign.loc[i,'자치구명'] = '금천구'
    elif time_sign.loc[i,'경찰서'] =='갈동':
        time_sign.loc[i,'자치구명'] = '강동구'
    elif time_sign.loc[i,'경찰서'] =='남대문':
        time_sign.loc[i,'자치구명'] = '중구'
    elif time_sign.loc[i,'경찰서'] =='중부':
        time_sign.loc[i,'자치구명'] = '중구'
    elif time_sign.loc[i,'경찰서'] =='동대문':
        time_sign.loc[i,'자치구명'] = '동대문구'
    elif time_sign.loc[i,'경찰서'] =='양천':
        time_sign.loc[i,'자치구명'] = '양천구'
    else:
        time_sign.loc[i,'자치구명'] = time_sign.loc[i,'경찰서']+'구'
# time_sign


##### 2번 전처리
###### 엑셀로 변환과정에서 생긴 글자 인식 오류 -> 올바른 글자로 수정(오류가 별로 없다 나중에)
##### 3. 운영시간 형식 수정 및 통일
###### 3-1. 평일 운영, 주말 운영 등 특징은 따로 컬럼으로 나타내주기"

In [54]:
#과정 복잡한 거 고려해서 새롭게 데이터 프레임 생성\n",
time_sign_t = time_sign[['연번','점멸시간']]
# time_sign_t['점멸시간'].unique()

##### "컬럼을 다음과 같이 구분
- 크게 매일, 평일, 주말로 구분
1. 평일 주말 구분 없이 매일 운영하는 경우
- 매일 1번 -> 매일1 컬럼
- 매일 2번 -> 매일1, 매일2 컬럼

2. 평일 - 평일과 주말을 나누어 운영하는경우
- 주말 항시 점멸 -> 토요일, 일요일 컬럼에 "00:00~24:00" 값 기입
3. 매일 한번, 월-토에 한 번 더 운영하는 겨우
- 매일 1 컬럼에 매일 운영하는 시간 기입
- 매일2, 토요일 컬럼에 한 번 더 운영하는 시간 기입
4. 토요일, 일요일, 특이사항
- 토요일만, 일요일만, 공휴일만 운영하는 경우 고려
5. 각 컬럼에 해당하는 값이 없는 경우는 널값이 아닌 0으로 둔다.


In [53]:
time_sign_t['점멸시간'] = time_sign_t['점멸시간'].str.replace('익일', '') # '익일' 제거
time_sign_t['점멸시간'] = time_sign_t['점멸시간'].str.replace('항시', '') # '항시' 제거
time_sign_t['점멸시간'] = time_sign_t['점멸시간'].str.replace('전일', '') # '전일' 제거
time_sign_t['점멸시간'] = time_sign_t['점멸시간'].str.replace(' ', '') # 띄어쓰기 제거
time_sign_t['점멸시간'] = time_sign_t['점멸시간'].str.replace('(', '') # ( 제거
time_sign_t['점멸시간'] = time_sign_t['점멸시간'].str.replace(')', '') # ) 제거
time_sign_t['점멸시간'] = time_sign_t['점멸시간'].str.replace('매일', '') # '매일' 제거
time_sign_t['점멸시간'] = time_sign_t['점멸시간'].str.replace('평일', '') #'평일' 제거
# time_sign_t['점멸시간'].unique()

In [23]:
#우선 별다른 특이사항 없는 값은 그대로 넣어줌
for i in range(len(time_sign_t)):
    if len(time_sign_t.loc[i,'점멸시간'])==11:
        time_sign_t.loc[i,'매일1'] = time_sign_t.loc[i,'점멸시간']
        
time_sign_t['매일1'].unique()

array(['00:00~04:00', '20:00~07:00', '00:00~05:00', '01:00~05:00',
       '01:00~04:00', '21:00~07:00', '00:00~24:00', '20:00~06:00',
       '15:00~07:30', '23:00~05:00', '22:00~05:00', nan, '20:00~08:00',
       '18:00~06:00', '08:00~09:00', '20:00~05:00', '09:30~06:00',
       '21:00~06:00', '17:00~19:00', '00:00~06:00', '17:00~06:00',
       '23:00~06:00', '19:00~07:00', '09:00~08:00', '22:00~07:30'],
      dtype=object)

In [24]:
#매일에 운영하는 시간대가 두번인 경우\
for i in range(len(time_sign)):
    if time_sign_t.loc[i,'점멸시간']=='15:00~07:30\n09:00~12:00':
        time_sign_t.loc[i,'매일1'] = time_sign_t.loc[i,'점멸시간'][:11]
        time_sign_t.loc[i,'매일2'] = time_sign_t.loc[i,'점멸시간'][12:]
    elif time_sign_t.loc[i,'점멸시간']== '00:00~05:00\n11:30~14:00':
        time_sign_t.loc[i,'매일1'] = time_sign_t.loc[i,'점멸시간'][:11]
        time_sign_t.loc[i,'매일2'] = time_sign_t.loc[i,'점멸시간'][12:]
    else:
        pass
time_sign_t['매일2'].unique()

array([nan, '09:00~12:00', '11:30~14:00'], dtype=object)

##### 평일 ~ 주말엔 항시 점멸 경우
'18:00~07:00\n주말점멸'
'00:00~05:00\n토,일점멸'
'17:30~07:00\n주말:점멸'
':19:00~07:00\n주말:점멸'
':17:00~07:00\n주말:점멸'
'20:00~07:00\n주말점멸'
'17:00~07:00\n주말점멸'
':21:00~08:00\n그외점멸'


In [25]:
#평일 ~ 주말엔 항시 점멸 경우

for i in range(len(time_sign_t)):
    if '주말' in time_sign_t.loc[i,'점멸시간'] and '점멸' in time_sign_t.loc[i,'점멸시간']:
        if time_sign_t.loc[i,'점멸시간'][0]!=':' :
            time_sign_t.loc[i,'평일'] = time_sign_t.loc[i,'점멸시간'][:11]
            time_sign_t.loc[i,'토요일'] = '00:00~24:00'
            time_sign_t.loc[i,'일요일'] = '00:00~24:00'
        else:
            time_sign_t.loc[i,'평일'] = time_sign_t.loc[i,'점멸시간'][1:12]
            time_sign_t.loc[i,'토요일'] = '00:00~24:00'
            time_sign_t.loc[i,'일요일'] = '00:00~24:00'
time_sign_t['평일'].unique()   

array([nan, '18:00~07:00', '17:30~07:00', '19:00~07:00', '17:00~07:00',
       '20:00~07:00'], dtype=object)

In [26]:
#'00:00~05:00\\n토,일점멸'\n",
#':21:00~08:00\\n그외점멸'\n",
for i in range(len(time_sign_t)):
    if time_sign_t.loc[i,'점멸시간']=='00:00~05:00\n토,일점멸' :
        time_sign_t.loc[i,'평일'] = time_sign_t.loc[i,'점멸시간'][:11]
        time_sign_t.loc[i,'토요일'] = '00:00~24:00'
        time_sign_t.loc[i,'일요일'] = '00:00~24:00'
    elif time_sign_t.loc[i,'점멸시간']==':21:00~08:00\n그외점멸':
        time_sign_t.loc[i,'평일'] = time_sign_t.loc[i,'점멸시간'][1:12]
        time_sign_t.loc[i,'토요일'] = '00:00~24:00'
        time_sign_t.loc[i,'일요일'] = '00:00~24:00'
    else:       
        pass
time_sign_t['평일'].unique()

array([nan, '18:00~07:00', '00:00~05:00', '17:30~07:00', '19:00~07:00',
       '17:00~07:00', '20:00~07:00', '21:00~08:00'], dtype=object)

In [28]:
#남은 전처리 값 확인\n",
# 어떤 컬럼에도 값이 없는 행 확인\n",
time_sign_t[time_sign_t[['매일1' ,'평일','토요일', '일요일']].isnull().all(axis=1)]

,연번,점멸시간,매일1,매일2,평일,토요일,일요일
403,404,00:00~05:00\n11:00~17:00월~토,NaN,NaN,NaN,NaN,NaN
483,484,토요일14:00~00:00\n일요일00:00~22:00공휴일10:00~22:00,NaN,NaN,NaN,NaN,NaN
624,625,00:00~05:00\n주말00:00~07:00,NaN,NaN,NaN,NaN,NaN
625,626,00:00~05:00\n주말00:00~07:00,NaN,NaN,NaN,NaN,NaN
1013,1014,"07:30~17:00\n일요일,특수일점멸",NaN,NaN,NaN,NaN,NaN


In [32]:
time_sign_t.loc[403,'매일1'],time_sign_t.loc[403,'평일'],time_sign_t.loc[403,'토요일'] = time_sign_t.loc[403,'점멸시간'][:11],time_sign_t.loc[403,'점멸시간'][12:23],time_sign_t.loc[403,'점멸시간'][12:23]
time_sign_t.loc[483,'토요일'],time_sign_t.loc[483,'일요일'],time_sign_t.loc[483,'특이사항'] = time_sign_t.loc[483,'점멸시간'][3:14],time_sign_t.loc[483,'점멸시간'][18:29],time_sign_t.loc[483,'점멸시간'][29:]
time_sign_t.loc[624,'매일1'],time_sign_t.loc[624,'토요일'],time_sign_t.loc[624,'일요일'] =time_sign_t.loc[624,'점멸시간'][:11],time_sign_t.loc[624,'점멸시간'][14:],time_sign_t.loc[624,'점멸시간'][14:]
time_sign_t.loc[625,'매일1'],time_sign_t.loc[625,'토요일'],time_sign_t.loc[625,'일요일'] =time_sign_t.loc[625,'점멸시간'][:11],time_sign_t.loc[625,'점멸시간'][14:],time_sign_t.loc[625,'점멸시간'][14:]
time_sign_t.loc[1013,'평일'],time_sign_t.loc[1013,'토요일'],time_sign_t.loc[1013,'일요일'],time_sign_t.loc[1013,'특이사항']=time_sign_t.loc[1013,'점멸시간'][:11],time_sign_t.loc[1013,'점멸시간'][:11],'00:00~24:00',time_sign_t.loc[1013,'점멸시간'][-5:-2]+'00:00~24:00'

In [33]:
print(time_sign_t['매일1'].unique())
print(time_sign_t['매일2'].unique())
print(time_sign_t['평일'].unique())
print(time_sign_t['토요일'].unique())
print(time_sign_t['일요일'].unique())
print(time_sign_t['특이사항'].unique())


['00:00~04:00' '20:00~07:00' '00:00~05:00' '01:00~05:00' '01:00~04:00'
 '21:00~07:00' '00:00~24:00' '20:00~06:00' '15:00~07:30' '23:00~05:00'
 '22:00~05:00' nan '20:00~08:00' '18:00~06:00' '08:00~09:00' '20:00~05:00'
 '09:30~06:00' '21:00~06:00' '17:00~19:00' '00:00~06:00' '17:00~06:00'
 '23:00~06:00' '19:00~07:00' '09:00~08:00' '22:00~07:30']
[nan '09:00~12:00' '11:30~14:00']
[nan '18:00~07:00' '11:00~17:00' '00:00~05:00' '17:30~07:00' '19:00~07:00'
 '17:00~07:00' '20:00~07:00' '21:00~08:00' '07:30~17:00']
[nan '00:00~24:00' '11:00~17:00' '14:00~00:00' '00:00~07:00' '07:30~17:00']
[nan '00:00~24:00' '00:00~22:00' '00:00~07:00']
[nan '공휴일10:00~22:00' '특수일00:00~24:00']


In [35]:
#최종적으로 널값에 0부여
time_sign_t = time_sign_t.fillna(0)
print(time_sign_t['매일1'].unique())
print(time_sign_t['매일2'].unique())
print(time_sign_t['평일'].unique())
print(time_sign_t['토요일'].unique())
print(time_sign_t['일요일'].unique())
print(time_sign_t['특이사항'].unique())

['00:00~04:00' '20:00~07:00' '00:00~05:00' '01:00~05:00' '01:00~04:00'
 '21:00~07:00' '00:00~24:00' '20:00~06:00' '15:00~07:30' '23:00~05:00'
 '22:00~05:00' 0 '20:00~08:00' '18:00~06:00' '08:00~09:00' '20:00~05:00'
 '09:30~06:00' '21:00~06:00' '17:00~19:00' '00:00~06:00' '17:00~06:00'
 '23:00~06:00' '19:00~07:00' '09:00~08:00' '22:00~07:30']
[0 '09:00~12:00' '11:30~14:00']
[0 '18:00~07:00' '11:00~17:00' '00:00~05:00' '17:30~07:00' '19:00~07:00'
 '17:00~07:00' '20:00~07:00' '21:00~08:00' '07:30~17:00']
[0 '00:00~24:00' '11:00~17:00' '14:00~00:00' '00:00~07:00' '07:30~17:00']
[0 '00:00~24:00' '00:00~22:00' '00:00~07:00']
[0 '공휴일10:00~22:00' '특수일00:00~24:00']


In [46]:
#예시로 한 행 값 확인
res_time_sign = pd.concat([time_sign[['교차로명','자치구명']],time_sign_t[['매일1','매일2','평일','토요일','일요일','특이사항']]],axis=1)
res_time_sign.iloc[624]

교차로명     관악드림타운107동
자치구명            관악구
매일1     00:00~05:00
매일2               0
평일                0
토요일     00:00~07:00
일요일     00:00~07:00
특이사항              0
Name: 624, dtype: object

In [55]:
# # 데이터프레임을 CSV 파일로 저장
file_path = '/Users/kyuree/Desktop/pds_capstone1/processed_dataset/kr_자치구별_점멸신호_운영시간대_수정.csv'
res_time_sign.to_csv(file_path, index=False)
res_time_sign
   

In [ ]:
#5. 자치구별 운영 점멸신호 개수 파악"

In [48]:
#시간제 점멸 자치구별 개수 확인\n",
time_sign['자치구명'].value_counts()
time_sign_cnt = pd.DataFrame(time_sign['자치구명'].value_counts().sort_index())
time_sign_cnt.columns=['운영개수']
time_sign_cnt

,운영개수
강남구,34
강동구,61
강북구,28
강서구,94
관악구,28
광진구,35
구로구,40
금천구,15
노원구,98
도봉구,43


In [49]:
#항시 점멸 신호기 자치구별 운영개수 확인\n",
always_sign_cnt = pd.DataFrame(always_sign['자치구명'].value_counts())

always_sign_cnt.loc['금천구']=0
always_sign_cnt.loc['동대문구']=0
always_sign_cnt.loc['양천구']=0
always_sign_cnt.loc['중구']=0
always_sign_cnt.columns=['운영개수']
always_sign_cnt=always_sign_cnt.sort_index()
always_sign_cnt
   

,운영개수
강남구,10
강동구,5
강북구,1
강서구,17
관악구,1
광진구,4
구로구,5
금천구,0
노원구,5
도봉구,3


In [52]:
#최종 자치구별 점멸신호 운영개수 확인,
result_df = pd.DataFrame({'모든 점멸신호 운영개수': always_sign_cnt['운영개수'] + time_sign_cnt['운영개수']}).reset_index().rename(columns={'index':'자치구명'})
    
result_df

,자치구명,모든 점멸신호 운영개수
0,강남구,44
1,강동구,66
2,강북구,29
3,강서구,111
4,관악구,29
5,광진구,39
6,구로구,45
7,금천구,15
8,노원구,103
9,도봉구,46


In [ ]:
# 데이터프레임을 CSV 파일로 저장\n",
result_df.to_csv('kr_자치구별_점멸신호현황.csv', index=False)